In [1]:
import polars as pl
import pandas as pd
from IPython.display import display

In [2]:
from Predictors.Pb.features import validate_features_vs_target

In [1]:
import Predictors.DataPrep as dp
results_with_pb_df = dp.get_data()

display(results_with_pb_df)

Position,Time,time_in_seconds,Bib Number,Name,Gender,Gender Position,Club,is_club_member,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_run_attempts,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_time_in_seconds,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,prior_quickest_vs_run_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_year,position_record_worst_year,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob
i64,str,f64,i64,str,str,f64,str,bool,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64
3,"""00:29:14""",1754.0,880,"""Andrew Masters""","""M""",3.0,"""Lewes AC""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:39.250000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,24.0,16.0,8.0,12.0,12.0,12.0,10.0,10.0,10.0,1.0,1.0,1.0,0.0,0.0,2014.0,2014.0,12.0,12.0,12.0,"""Andrew""","""Masters""","""V50""","""V50""","""V50""","""M""","""Lewes/West 4/Wimbledon Windmil…",80881.0,"""match""",1.0,"""all""",40.0,44.0,42.0,1974.0,1970.0,1972.0
4,"""00:29:34""",1774.0,588,"""Paul Cousins""","""M""",4.0,"""Haywards Heath Harriers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:41.750000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,9.0,5.0,4.0,2.0,5.0,7.0,10.0,10.0,10.0,3.0,3.0,1.0,0.0,0.0,2014.0,2014.0,2.0,5.0,7.0,"""Paul""","""Cousins""","""V60""","""V60""","""V60""","""M""","""Haywards Heath""",9998.0,"""match""",1.0,"""all""",50.0,54.0,52.0,1964.0,1960.0,1962.0
5,"""00:29:47""",1787.0,902,"""Michael Neville""","""M""",5.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:43.375000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Michael""","""Neville""","""V45""","""V45""","""V45""","""M""","""Chichester""",43629.0,"""match""",1.0,"""all""",35.0,39.0,37.0,1979.0,1975.0,1977.0
6,"""00:30:07""",1807.0,918,"""Conrad Meagher""","""M""",6.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:45.875000""",2.0,2.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Conrad""","""Meagher""","""SEN""","""SEN""","""SEN""","""M""","""Havant""",566746.0,"""match""",1.0,"""no_club""",13.0,24.0,18.5,2001.0,1990.0,1995.5
7,"""00:30:09""",1809.0,530,"""John Russell""","""M""",7.0,"""Portslade Hedgehoppers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:46.125000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,23.0,15.0,8.0,10.0,27.0,72.0,10.0,9.0,8.0,10.0,10.0,1.0,0.0,0.0,2014.0,2014.0,10.0,27.0,72.0,null,null,"""TBC""","""TBC""","""TBC""","""n""",null,null,"""missing""",0.0,"""not found""",null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",2861.0,631,"""Ann Savidge""","""F""",303.0,"""Burgess 